# Recommender Systems - Data Mining Lab 2

In [2]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

### Data loading

In [3]:
train_file_path = 'data/lab2_train.csv' 
test_file_path = 'data/lab2_test.csv'

train_data = pd.read_csv(train_file_path, delimiter=',')
test_data = pd.read_csv(test_file_path, delimiter=',')


In [45]:
#print(train_data)
like_train_data = train_data[train_data.is_like == True]
like_train_data = like_train_data.sort_values("user_to_id")
match_train_data = train_data[train_data.is_match ==True]
match_train_data = match_train_data.sort_values("user_from_id")

liked_user_count = like_train_data["user_to_id"].nunique()
user_count = like_train_data["user_from_id"].nunique()
match_count = match_train_data["user_from_id"].nunique()

print(user_count)
print(liked_user_count)
print(match_count)
# User 2 receives a lot of likes but doesn't like
print(like_train_data[lik])
#print(like_train_data[like_train_data.user_to_id == 2])
#print(train_data[train_data.user_from_id == 2])


#print(like_train_data)

#print(match_train_data)

2107
1707
345
       user_from_id  user_to_id  is_like is_match
50598            93           2     True    False
46348            24           2     True    False
11643            88           2     True    False
61229            82           2     True    False
71280            67           2     True    False
11344            74           2     True    False
1648             16           2     True    False
50638            90           2     True    False
27371            47           2     True    False
25066            41           2     True    False
62824            25           2     True    False
25395            39           2     True    False
51239            62           2     True    False
51374            77           2     True    False
26994            64           2     True    False
56399            19           2     True    False
52013            66           2     True    False
7813             20           2     True    False
55281            27           2     

## Familiarization
### Important properties of the data

### Types of people
There are only 1707 user who have one like or more of the 2107 users who gave likes. Of all of these users there are only 415 matches. Of these matches there are 345 unique users who receive a match. So there is only a small group of users who receive more than one match.

- Someone who receives a lot of likes but likes almost no one. User 2 receives 45 likes but likes none of them. Therefore not leading to a match

- Someone who receives about equal number of likes as he gives. User 